Gradient Boosting Steps/Processes:

Initial prediction (Mean of the target)
Residual = Target - Prediction trees= []
For loop (loop for 1000's)
Create a decision trees => tarin residual
train(x, residuals)
predicitons = predict(x) => predict residual
Residual = Residual - learning_rate * prediction
trees.append(tree)
1.Initial prediction => mean of target 2. REsidual = target- prediction 3. train decision tree = train residual 4. prediciton => prediciton of residual 5. new prediciton = old prediciton + LR * prediciton 6.step 2 for all thre trees